In [40]:
import requests
import pprint
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# make a 'get' request
res = requests.get("https://api.flightstats.com/flex/schedules/rest/v1/json/from/lax/to/jfk/departing/2019/10/18?appId=c1272abb&appKey=58dfa6e47bc7ec284869e36f94613a6f")


In [5]:
# print status code
print(res.status_code)

200


In [25]:
# print json
pprint.pprint(res.json(), depth=1)

{'appendix': {...}, 'flightStatuses': [...], 'request': {...}}


In [18]:
# store relevant data
raw = res.json()
lvl1_keys = []

for key in raw:
    lvl1_keys.append(key)
    
print(lvl1_keys)

['request', 'scheduledFlights', 'appendix']


In [24]:
# create new dataframe with desired data
flights = raw['scheduledFlights']
#pprint.pprint(flights)
df = pd.DataFrame.from_dict(flights)
df.head()

,arrivalAirportFsCode,arrivalTerminal,arrivalTime,carrierFsCode,codeshares,departureAirportFsCode,departureTerminal,departureTime,flightEquipmentIataCode,flightNumber,isCodeshare,isWetlease,operator,referenceCode,serviceClasses,serviceType,stops,trafficRestrictions
0,JFK,4,2019-10-19T06:47:00.000,DL,"[{'carrierFsCode': 'CZ', 'flightNumber': '1136...",LAX,2,2019-10-18T22:30:00.000,75W,1908,False,False,NaN,1013-1559052--,"[J, Y]",J,0,[]
1,JFK,4,2019-10-19T06:47:00.000,CZ,[],LAX,2,2019-10-18T22:30:00.000,75W,1136,True,False,"{'carrierFsCode': 'DL', 'flightNumber': '1908'...",1013-1559052--1302993,"[J, Y]",J,0,[G]
2,JFK,4,2019-10-19T06:47:00.000,KE,[],LAX,2,2019-10-18T22:30:00.000,75W,7537,True,False,"{'carrierFsCode': 'DL', 'flightNumber': '1908'...",1013-1559052--10952026,"[R, J, Y]",J,0,[Q]
3,JFK,4,2019-10-19T06:47:00.000,VA,[],LAX,2,2019-10-18T22:30:00.000,75W,6654,True,False,"{'carrierFsCode': 'DL', 'flightNumber': '1908'...",1013-1559052--10952028,"[J, Y]",J,0,[X]
4,JFK,4,2019-10-19T06:47:00.000,AZ,[],LAX,2,2019-10-18T22:30:00.000,75W,5897,True,False,"{'carrierFsCode': 'DL', 'flightNumber': '1908'...",1013-1559052--10952029,"[J, Y]",J,0,[G]


In [32]:
df.carrierFsCode.value_counts()

QF    12
B6    11
AA    11
VS    10
DL    10
VA    10
AZ    10
BA     9
SQ     7
TN     6
HA     6
AM     6
QR     6
KL     5
AS     5
KE     4
UX     4
GF     3
CZ     3
EK     3
AF     3
MU     3
IB     3
AY     2
SA     2
AT     2
EY     2
JJ     2
AR     2
G3     2
FJ     1
FI     1
JL     1
TP     1
Name: carrierFsCode, dtype: int64

In [34]:
# a different way to convert response object to make it easier to read
def json_print(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    
json_print(raw)

{
    "appendix": {
        "airlines": [
            {
                "active": true,
                "fs": "JJ",
                "iata": "JJ",
                "icao": "TAM",
                "name": "LATAM Airlines Brasil",
                "phoneNumber": "1-888-2FLY TAM"
            },
            {
                "active": true,
                "fs": "JL",
                "iata": "JL",
                "icao": "JAL",
                "name": "JAL",
                "phoneNumber": "0120-25-5931"
            },
            {
                "active": true,
                "fs": "FI",
                "iata": "FI",
                "icao": "ICE",
                "name": "Icelandair"
            },
            {
                "active": true,
                "fs": "FJ",
                "iata": "FJ",
                "icao": "FJI",
                "name": "Fiji Airways",
                "phoneNumber": "1-800-227-4446"
            },
            {
                "active": true,
             

In [38]:
flights_dump = json.dumps(flights, sort_keys=True, indent=4)
print(flights_dump)


[
    {
        "arrivalAirportFsCode": "JFK",
        "arrivalTerminal": "4",
        "arrivalTime": "2019-10-19T06:47:00.000",
        "carrierFsCode": "DL",
        "codeshares": [
            {
                "carrierFsCode": "CZ",
                "flightNumber": "1136",
                "referenceCode": 1302993,
                "serviceClasses": [
                    "J",
                    "Y"
                ],
                "serviceType": "J",
                "trafficRestrictions": [
                    "G"
                ]
            },
            {
                "carrierFsCode": "KE",
                "flightNumber": "7537",
                "referenceCode": 10952026,
                "serviceClasses": [
                    "R",
                    "J",
                    "Y"
                ],
                "serviceType": "J",
                "trafficRestrictions": [
                    "Q"
                ]
            },
            {
                "carrierFsCode

### Making a request using parameters

In [23]:
# the actual request url
# "https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/status/lax/dep/2019/10
# /11/0?appId=c1272abb&appKey=58dfa6e47bc7ec284869e36f94613a6f&utc=false&numHours=1&
# maxFlights=5"

def build_url(params):
    url = (
        f"{api_params['url']}/{api_params['airport']}/dep/{api_params['year']}"
        f"/{api_params['month']}/{api_params['day']}/{api_params['hour']}"
    )
    
    return url

api_params = {
    'url'    : 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/status',
    'airport': 'lax',
    'year'   : '2019',
    'month'  : '10',
    'day'    : '11',
    'hour'   : '6'
}

api_creds = {
    'appId' : 'c1272abb',
    'appKey': '58dfa6e47bc7ec284869e36f94613a6f'
}

res = requests.get(build_url(api_params), params=api_creds)


In [26]:
# print res body
pprint.pprint(res.json(), depth=1)

{'appendix': {...}, 'flightStatuses': [...], 'request': {...}}


In [30]:
pprint.pprint(res.json()['flightStatuses'], depth=2)

[{'airportResources': {...},
  'arrivalAirportFsCode': 'ORD',
  'arrivalDate': {...},
  'carrierFsCode': 'AA',
  'codeshares': [...],
  'delays': {...},
  'departureAirportFsCode': 'LAX',
  'departureDate': {...},
  'flightDurations': {...},
  'flightEquipment': {...},
  'flightId': 1017192342,
  'flightNumber': '296',
  'operationalTimes': {...},
  'schedule': {...},
  'status': 'L'},
 {'airportResources': {...},
  'arrivalAirportFsCode': 'HND',
  'arrivalDate': {...},
  'carrierFsCode': 'NH',
  'codeshares': [...],
  'delays': {...},
  'departureAirportFsCode': 'LAX',
  'departureDate': {...},
  'flightDurations': {...},
  'flightEquipment': {...},
  'flightId': 1017372181,
  'flightNumber': '105',
  'operationalTimes': {...},
  'schedule': {...},
  'status': 'U'},
 {'airportResources': {...},
  'arrivalAirportFsCode': 'HGH',
  'arrivalDate': {...},
  'carrierFsCode': '3U',
  'codeshares': [],
  'delays': {...},
  'departureAirportFsCode': 'LAX',
  'departureDate': {...},
  'flightDu

  'operationalTimes': {...},
  'schedule': {...},
  'status': 'L'},
 {'airportResources': {...},
  'arrivalAirportFsCode': 'SJD',
  'arrivalDate': {...},
  'carrierFsCode': 'AS',
  'codeshares': [...],
  'delays': {},
  'departureAirportFsCode': 'LAX',
  'departureDate': {...},
  'flightDurations': {...},
  'flightEquipment': {...},
  'flightId': 1017321347,
  'flightNumber': '208',
  'operationalTimes': {...},
  'schedule': {...},
  'status': 'L'},
 {'airportResources': {...},
  'arrivalAirportFsCode': 'SJC',
  'arrivalDate': {...},
  'carrierFsCode': 'AS',
  'codeshares': [...],
  'delays': {},
  'departureAirportFsCode': 'LAX',
  'departureDate': {...},
  'flightDurations': {...},
  'flightEquipment': {...},
  'flightId': 1017321257,
  'flightNumber': '1489',
  'operationalTimes': {...},
  'schedule': {...},
  'status': 'L'}]


In [32]:
data = res.json()['flightStatuses']
df = pd.DataFrame.from_dict(data)
df.head()

,airportResources,arrivalAirportFsCode,arrivalDate,carrierFsCode,codeshares,delays,departureAirportFsCode,departureDate,divertedAirportFsCode,flightDurations,flightEquipment,flightId,flightNumber,operationalTimes,schedule,status
0,"{'departureTerminal': '4', 'departureGate': '4...",ORD,"{'dateUtc': '2019-10-11T10:57:00.000Z', 'dateL...",AA,"[{'fsCode': 'AY', 'flightNumber': '4021', 'rel...","{'departureGateDelayMinutes': 579, 'departureR...",LAX,"{'dateUtc': '2019-10-11T06:55:00.000Z', 'dateL...",NaN,"{'scheduledBlockMinutes': 242, 'blockMinutes':...","{'scheduledEquipmentIataCode': '321', 'actualE...",1017192342,296,{'publishedDeparture': {'dateUtc': '2019-10-11...,"{'flightType': 'J', 'serviceClasses': 'RJY', '...",L
1,"{'departureTerminal': 'TBIT', 'arrivalTerminal...",HND,"{'dateUtc': '2019-10-11T20:00:00.000Z', 'dateL...",NH,"[{'fsCode': 'UA', 'flightNumber': '7983', 'rel...","{'departureGateDelayMinutes': 480, 'arrivalGat...",LAX,"{'dateUtc': '2019-10-11T07:50:00.000Z', 'dateL...",NaN,"{'scheduledBlockMinutes': 730, 'scheduledAirMi...","{'scheduledEquipmentIataCode': '77W', 'actualE...",1017372181,105,{'publishedDeparture': {'dateUtc': '2019-10-11...,"{'flightType': 'J', 'serviceClasses': 'FJY', '...",U
2,"{'departureTerminal': 'TBIT', 'arrivalTerminal...",HGH,"{'dateUtc': '2019-10-11T22:50:00.000Z', 'dateL...",3U,[],"{'departureGateDelayMinutes': 279, 'departureR...",LAX,"{'dateUtc': '2019-10-11T08:50:00.000Z', 'dateL...",NaN,"{'scheduledBlockMinutes': 840, 'scheduledAirMi...","{'scheduledEquipmentIataCode': '359', 'actualE...",1017301625,8632,{'publishedDeparture': {'dateUtc': '2019-10-11...,"{'flightType': 'J', 'serviceClasses': 'RFJY', ...",L
3,"{'departureTerminal': '4', 'departureGate': '4...",SAN,"{'dateUtc': '2019-10-11T12:59:00.000Z', 'dateL...",AA,[],"{'departureGateDelayMinutes': 118, 'departureR...",LAX,"{'dateUtc': '2019-10-11T12:00:00.000Z', 'dateL...",NaN,"{'scheduledBlockMinutes': 59, 'blockMinutes': ...","{'actualEquipmentIataCode': '321', 'tailNumber...",1017638176,596,{'scheduledGateDeparture': {'dateUtc': '2019-1...,NaN,L
4,{},RNO,"{'dateUtc': '2019-10-11T15:08:00.000Z', 'dateL...",AMF,[],{},LAX,"{'dateUtc': '2019-10-11T12:47:00.000Z', 'dateL...",NaN,{'airMinutes': 113},"{'actualEquipmentIataCode': 'BE9', 'tailNumber...",1017639706,1450,{'scheduledGateDeparture': {'dateUtc': '2019-1...,NaN,L


In [39]:
# testing accessibility
df.iloc[0].delays

{'departureGateDelayMinutes': 579,
 'departureRunwayDelayMinutes': 498,
 'arrivalGateDelayMinutes': 571,
 'arrivalRunwayDelayMinutes': 491}

In [42]:
json_normalize(data)

,airportResources.arrivalGate,airportResources.arrivalTerminal,airportResources.baggage,airportResources.departureGate,airportResources.departureTerminal,arrivalAirportFsCode,arrivalDate.dateLocal,arrivalDate.dateUtc,carrierFsCode,codeshares,...,operationalTimes.scheduledRunwayArrival.dateLocal,operationalTimes.scheduledRunwayArrival.dateUtc,operationalTimes.scheduledRunwayDeparture.dateLocal,operationalTimes.scheduledRunwayDeparture.dateUtc,schedule.downlines,schedule.flightType,schedule.restrictions,schedule.serviceClasses,schedule.uplines,status
0,H5,3,7,46C,4,ORD,2019-10-11T05:57:00.000,2019-10-11T10:57:00.000Z,AA,"[{'fsCode': 'AY', 'flightNumber': '4021', 'rel...",...,2019-10-11T07:05:00.000,2019-10-11T12:05:00.000Z,2019-10-11T01:36:00.000,2019-10-11T08:36:00.000Z,[],J,,RJY,[],L
1,NaN,INTL,NaN,NaN,TBIT,HND,2019-10-12T05:00:00.000,2019-10-11T20:00:00.000Z,NH,"[{'fsCode': 'UA', 'flightNumber': '7983', 'rel...",...,2019-10-12T03:59:00.000,2019-10-11T18:59:00.000Z,2019-10-11T01:24:00.000,2019-10-11T08:24:00.000Z,[],J,,FJY,[],U
2,NaN,2,NaN,NaN,TBIT,HGH,2019-10-12T06:50:00.000,2019-10-11T22:50:00.000Z,3U,[],...,2019-10-12T07:21:00.000,2019-10-11T23:21:00.000Z,2019-10-11T02:02:00.000,2019-10-11T09:02:00.000Z,"[{'fsCode': 'CTU', 'flightId': 1017301634}]",J,,RFJY,[],L
3,29,2,NaN,47B,4,SAN,2019-10-11T05:59:00.000,2019-10-11T12:59:00.000Z,AA,[],...,2019-10-11T05:43:00.000,2019-10-11T12:43:00.000Z,2019-10-11T05:16:00.000,2019-10-11T12:16:00.000Z,NaN,NaN,NaN,NaN,NaN,L
4,NaN,NaN,NaN,NaN,NaN,RNO,2019-10-11T08:08:00.000,2019-10-11T15:08:00.000Z,AMF,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L
5,1,NaN,NaN,15,1,DAL,2019-10-11T10:55:00.000,2019-10-11T15:55:00.000Z,WN,[],...,2019-10-11T10:35:00.000,2019-10-11T15:35:00.000Z,2019-10-11T06:06:00.000,2019-10-11T13:06:00.000Z,[],J,F,RFJY,[],L
6,NaN,1,NaN,69B,6,YYZ,2019-10-11T13:41:00.000,2019-10-11T17:41:00.000Z,AC,"[{'fsCode': 'NZ', 'flightNumber': '4790', 'rel...",...,2019-10-11T13:18:00.000,2019-10-11T17:18:00.000Z,2019-10-11T06:06:00.000,2019-10-11T13:06:00.000Z,[],J,,RFJY,[],L
7,13,N,14,67,6,SEA,2019-10-11T09:00:00.000,2019-10-11T16:00:00.000Z,AS,"[{'fsCode': 'KE', 'flightNumber': '6106', 'rel...",...,2019-10-11T08:33:00.000,2019-10-11T15:33:00.000Z,2019-10-11T06:21:00.000,2019-10-11T13:21:00.000Z,[],J,,RFJY,[],L
8,D5,NaN,NaN,24,2,PDX,2019-10-11T08:30:00.000,2019-10-11T15:30:00.000Z,DL,"[{'fsCode': 'KE', 'flightNumber': '6815', 'rel...",...,2019-10-11T08:17:00.000,2019-10-11T15:17:00.000Z,2019-10-11T06:20:00.000,2019-10-11T13:20:00.000Z,"[{'fsCode': 'LAX', 'flightId': 1017336135}]",J,,JY,[],L
9,NaN,NaN,NaN,NaN,NaN,MCO,2019-10-11T13:35:00.000,2019-10-11T17:35:00.000Z,K4,[],...,NaN,NaN,NaN,NaN,[],F,,,[],L
